In [5]:
import pandas as pd

sleep_quality_dataset = pd.read_csv('data/sleep_score_data_fitbit.csv')

print(sleep_quality_dataset.shape)
sleep_quality_dataset.head()

(291, 6)


,timestamp,overall_score,revitalization_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
0,2023-11-18T07:59:00Z,83,83,104,63,0.068100
1,2023-11-17T09:14:00Z,87,87,114,63,0.053283
2,2023-11-16T06:01:30Z,84,84,99,64,0.051408
3,2023-11-15T05:23:00Z,81,81,73,65,0.046679
4,2023-11-14T05:54:30Z,76,76,64,65,0.076923


In [6]:
sleep_quality_dataset['timestamp'] = pd.to_datetime(
    sleep_quality_dataset['timestamp']
)
sleep_quality_dataset = sleep_quality_dataset.set_index('timestamp').sort_index()

In [7]:
sleep_quality_dataset['DayOfWeek'] = sleep_quality_dataset.index.dayofweek

sleep_quality_dataset['IsWeekend'] = sleep_quality_dataset['DayOfWeek'].apply(
    lambda x: 1 if x >= 5 else 0
)

sleep_quality_dataset['WakeupHour'] = sleep_quality_dataset.index.hour

sleep_quality_dataset['Score_Lag1'] = sleep_quality_dataset['overall_score'].shift(1)

sleep_quality_dataset['DeepSleep_Lag1'] = sleep_quality_dataset['deep_sleep_in_minutes'].shift(1)

sleep_quality_dataset['RHR_Lag1'] = sleep_quality_dataset['resting_heart_rate'].shift(1)

sleep_quality_dataset = sleep_quality_dataset.dropna()

print(sleep_quality_dataset.columns)



Index(['overall_score', 'revitalization_score', 'deep_sleep_in_minutes',
       'resting_heart_rate', 'restlessness', 'DayOfWeek', 'IsWeekend',
       'WakeupHour', 'Score_Lag1', 'DeepSleep_Lag1', 'RHR_Lag1'],
      dtype='object')


In [8]:
sleep_quality_dataset.head()

,overall_score,revitalization_score,deep_sleep_in_minutes,resting_heart_rate,restlessness,DayOfWeek,IsWeekend,WakeupHour,Score_Lag1,DeepSleep_Lag1,RHR_Lag1
timestamp,,,,,,,,,,,
2022-12-24 10:34:30+00:00,74,74,96,65,0.085350,5,1,10,78.0,91.0,65.0
2022-12-25 10:35:30+00:00,83,83,126,65,0.075336,6,1,10,74.0,96.0,65.0
2022-12-26 10:20:30+00:00,71,71,47,66,0.062583,0,0,10,83.0,126.0,65.0
2022-12-27 10:14:30+00:00,73,73,75,67,0.074119,1,0,10,71.0,47.0,66.0
2022-12-28 09:37:30+00:00,68,68,67,67,0.057718,2,0,9,73.0,75.0,67.0


In [10]:
sleep_quality_dataset.to_csv('data/sleep_quality_preprocessed.csv', index=True)

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score
import joblib

DATA_DIR = 'data'
MODEL_NAME_REGRESSION = 'random_forest_regression_model.pkl'

df_regression = pd.read_csv(os.path.join(DATA_DIR, 'sleep_quality_preprocessed.csv'), index_col=0)

TARGET_COL = 'overall_score'
EXCLUDE_COLS = [TARGET_COL, 'timestamp'] 
X = df_regression.drop(columns=EXCLUDE_COLS, errors='ignore')
Y = df_regression[TARGET_COL]

numerical_cols = X.select_dtypes(include=np.number).columns
imputer_reg = SimpleImputer(missing_values=np.nan, strategy='median')
X_imputed_reg = imputer_reg.fit_transform(X[numerical_cols])

X_processed_reg = pd.DataFrame(X_imputed_reg, columns=numerical_cols)


X_train_reg, X_test_reg, Y_train_reg, Y_test_reg = train_test_split(
    X_processed_reg, Y, test_size=0.2, random_state=42
)

rf_regressor = RandomForestRegressor(
    n_estimators=100,
    max_depth=8,
    random_state=42
)

print("\nStarting Random Forest Regressor training (Sleep Score)...")
rf_regressor.fit(X_train_reg, Y_train_reg)
print("Training complete.")


Y_pred_reg = rf_regressor.predict(X_test_reg)
mae = mean_absolute_error(Y_test_reg, Y_pred_reg)
r2 = r2_score(Y_test_reg, Y_pred_reg)

print("\n--- Sleep Score Model Evaluation (Random Forest Regressor) ---")
print(f"Mean Absolute Error (MAE): {mae:.2f} points (Average error)")
print(f"R-squared (R2) Score: {r2:.2f} (Model fit)")


joblib.dump(rf_regressor, MODEL_NAME_REGRESSION)
joblib.dump(imputer_reg, 'imputer_reg.pkl')

print(f"\n Regression Model saved: {MODEL_NAME_REGRESSION}")


Starting Random Forest Regressor training (Sleep Score)...
Training complete.

--- Sleep Score Model Evaluation (Random Forest Regressor) ---
Mean Absolute Error (MAE): 0.26 points (Average error)
R-squared (R2) Score: 0.97 (Model fit)

 Regression Model saved: random_forest_regression_model.pkl


: 

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report
import joblib

DATA_DIR = 'data'
MODEL_NAME_CLOUD = 'random_forest_sleep_classifier.pkl'

# --- Step 0: Load DataFrames (EXCLUDING WEIGHT AND CALORIES) ---
df_activity = pd.read_csv(os.path.join(DATA_DIR, 'dailyActivity_merged.csv'))
# df_calories is now excluded
df_intensity = pd.read_csv(os.path.join(DATA_DIR, 'dailyIntensities_merged.csv'))
df_sleep_day = pd.read_csv(os.path.join(DATA_DIR, 'sleepDay_merged.csv'))
df_sleep_min = pd.read_csv(os.path.join(DATA_DIR, 'minuteSleep_merged.csv'))
# here used to be df_weight but sadly this led to many nans after merging (this could have been a cool feature)

# 0.1 Standardize column names
df_activity.rename(columns={'ActivityDate': 'Date'}, inplace=True)
df_sleep_day.rename(columns={'SleepDay': 'Date'}, inplace=True)
df_intensity.rename(columns={'ActivityDay': 'Date'}, inplace=True)

# 0.2 Convert all date columns to datetime objects
date_format = '%m/%d/%Y' 

# List of DataFrames to process (df_calories and df_weight removed)  -> NOTE: the following 10ths of lines might be inproper merging
dfs_to_process = [df_activity, df_intensity, df_sleep_day]

for df in dfs_to_process:
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], format=date_format, errors='coerce')


# 1.4 Merge Daily Logs on 'Id' and 'Date'
df_merged = df_activity
# Merge with df_intensity and df_sleep_day only
for df in [df_intensity, df_sleep_day]:
    df_merged = pd.merge(df_merged, df, on=['Id', 'Date'], how='outer', suffixes=('', '_y'))
    df_merged = df_merged.loc[:,~df_merged.columns.duplicated()]

# --- process intraday sleep data ---
df_sleep_min.rename(columns={'date': 'datetime', 'value': 'Duration', 'sleepState': 'State'}, inplace=True)
df_sleep_min['datetime'] = pd.to_datetime(df_sleep_min['datetime'], format='mixed', errors='coerce')
df_sleep_min['Date'] = df_sleep_min['datetime'].dt.normalize()

# aggregate minutes spent in Awake and Restless states
df_sleep_agg = df_sleep_min.groupby(['Id', 'Date', 'State']).size().reset_index(name='Minutes')
df_sleep_pivot = df_sleep_agg.pivot_table(
    index=['Id', 'Date'], 
    columns='State', 
    values='Minutes', 
    fill_value=0
).reset_index()

df_sleep_pivot.rename(
    columns={'Awake': 'MinutesAwake_Intraday', 'Restless': 'MinutesRestless_Intraday'}, 
    inplace=True
)

# merge the aggregated intraday data into the main file
df_merged = pd.merge(
    df_merged, 
    df_sleep_pivot[['Id', 'Date', 'MinutesAwake_Intraday', 'MinutesRestless_Intraday']], 
    on=['Id', 'Date'], 
    how='left'
)

# --- doing some feature engenieruing feature engineering ---

# sort and group by id and date for lagged features
df_merged = df_merged.sort_values(by=['Id', 'Date']).reset_index(drop=True)

# handle timebased features
df_merged['DayOfWeek'] = df_merged['Date'].dt.dayofweek
df_merged['IsWeekend'] = df_merged['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

# lagged Features (Lag-1)
# Removed BMI from lag_cols -> have almost all values missing (tho i don't know if this is because i merged them inproperly)
lag_cols = ['TotalSteps', 'TotalMinutesAsleep', 'Calories', 'VeryActiveMinutes']
for col in lag_cols:
    df_merged[f'{col}_Lag1'] = df_merged.groupby('Id')[col].shift(1)

# target Variable (Y): Sleep Quality Classification (Poor/Fair/Good)
def classify_sleep(row):
    if pd.isna(row['TotalTimeInBed']) or row['TotalTimeInBed'] == 0:
        return 'Poor'
        
    efficiency = row['TotalMinutesAsleep'] / row['TotalTimeInBed']
    
    if row['TotalMinutesAsleep'] >= 450 and efficiency >= 0.90:
        return 'Good'
    elif row['TotalMinutesAsleep'] < 360 or efficiency < 0.75:
        return 'Poor'
    else:
        return 'Fair'

df_merged['Sleep_Quality_Class'] = df_merged.apply(classify_sleep, axis=1)

# --- clean up nan rown
df_final = df_merged.dropna(subset=['TotalSteps_Lag1', 'TotalMinutesAsleep'])


# select features for final output (Removed WeightKg, BMI, BMI_Lag1)
final_cols = ['Id', 'Date', 'TotalSteps', 'TotalMinutesAsleep', 'TotalTimeInBed', 'MinutesAwake_Intraday', 
              'MinutesRestless_Intraday', 'Calories', 'VeryActiveMinutes', 
              'SedentaryMinutes', 'DayOfWeek', 'IsWeekend', 'TotalSteps_Lag1', 'TotalMinutesAsleep_Lag1', 
              'Calories_Lag1', 'VeryActiveMinutes_Lag1', 'Sleep_Quality_Class']

df_final = df_final[final_cols]
df_final.to_csv('fitbit_massive_training_dataset_clean.csv', index=False)


# --- here are the steps for training a model using our dataset from kaggle (steps are layered out so making a pipeline later would be easier) ---

# --- 1. extract x and y ---
EXCLUDE_COLS = ['Id', 'Date', 'Sleep_Quality_Class']
X = df_final.drop(columns=EXCLUDE_COLS)
Y = df_final['Sleep_Quality_Class']

# --- 2. must encode target col (Y) ---
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)

# --- 3. handle of missing values and nans (sadly this causes us to loose a lot of data after merging (there may be a better way of doing it but am not sure how or if)---
numerical_cols = X.select_dtypes(include=np.number).columns
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
X_imputed = imputer.fit_transform(X[numerical_cols])

X_processed = pd.DataFrame(X_imputed, columns=numerical_cols)


# --- 4. classic train test split ---
X_train, X_test, Y_train, Y_test = train_test_split(
    X_processed, Y_encoded, test_size=0.2, random_state=42, stratify=Y_encoded
)

# --- 5. simple model as neural networks are inappropriate in my opinion (because of the lack of data) ---
rf_model = RandomForestClassifier(
    n_estimators=100,      
    max_depth=10,          
    random_state=42,
    class_weight='balanced'
)

print("\nStarting Random Forest training on the CLEANED data...")
rf_model.fit(X_train, Y_train)
print("Training complete.")


# --- 6. eval model
Y_pred = rf_model.predict(X_test)
print("\n--- Cloud Model Evaluation (Random Forest, No Weight/BMI) ---")
print(classification_report(
    Y_test, Y_pred, 
    target_names=le.classes_
))


# --- 7. save this all for azure deployement later
joblib.dump(rf_model, MODEL_NAME_CLOUD)

# save imputer and label encoder
joblib.dump(imputer, 'imputer.pkl')
joblib.dump(le, 'label_encoder.pkl')

print(f"\n Cloud Model and Preprocessors saved: {MODEL_NAME_CLOUD}, imputer.pkl, label_encoder.pkl")
print("The training process is complete, and the model is ready for Azure deployment.")


Starting Random Forest training on the CLEANED data...
Training complete.

--- Cloud Model Evaluation (Random Forest, No Weight/BMI) ---
              precision    recall  f1-score   support

        Fair       1.00      0.96      0.98        25
        Good       0.91      1.00      0.95        21
        Poor       1.00      0.93      0.97        15

    accuracy                           0.97        61
   macro avg       0.97      0.96      0.97        61
weighted avg       0.97      0.97      0.97        61


 Cloud Model and Preprocessors saved: random_forest_sleep_classifier.pkl, imputer.pkl, label_encoder.pkl
The training process is complete, and the model is ready for Azure deployment.


In [ ]:
fitbit_massive_training_dataset = pd.read_csv('fitbit_massive_training_dataset.csv')

print(fitbit_massive_training_dataset.info())

fitbit_massive_training_dataset.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        303 non-null    int64  
 1   Date                      283 non-null    object 
 2   TotalSteps                283 non-null    float64
 3   TotalMinutesAsleep        303 non-null    float64
 4   TotalTimeInBed            303 non-null    float64
 5   MinutesAwake_Intraday     283 non-null    float64
 6   MinutesRestless_Intraday  283 non-null    float64
 7   WeightKg                  5 non-null      float64
 8   BMI                       5 non-null      float64
 9   Calories                  283 non-null    float64
 10  VeryActiveMinutes         283 non-null    float64
 11  SedentaryMinutes          283 non-null    float64
 12  DayOfWeek                 283 non-null    float64
 13  IsWeekend                 303 non-null    int64  
 14  TotalSteps

,Id,Date,TotalSteps,TotalMinutesAsleep,TotalTimeInBed,MinutesAwake_Intraday,MinutesRestless_Intraday,WeightKg,BMI,Calories,VeryActiveMinutes,SedentaryMinutes,DayOfWeek,IsWeekend,TotalSteps_Lag1,TotalMinutesAsleep_Lag1,BMI_Lag1,Calories_Lag1,VeryActiveMinutes_Lag1,Sleep_Quality_Class
0,1503960366,2016-04-13,10735.0,384.0,407.0,12.0,11.0,NaN,NaN,1797.0,21.0,776.0,2.0,0,13162.0,327.0,NaN,1985.0,25.0,Fair
1,1503960366,2016-04-15,9762.0,412.0,442.0,8.0,22.0,NaN,NaN,1745.0,29.0,726.0,4.0,0,10460.0,NaN,NaN,1776.0,30.0,Fair
2,1503960366,2016-04-16,12669.0,340.0,367.0,8.0,20.0,NaN,NaN,1863.0,36.0,773.0,5.0,1,9762.0,412.0,NaN,1745.0,29.0,Poor
3,1503960366,2016-04-17,9705.0,700.0,712.0,0.0,11.0,NaN,NaN,1728.0,38.0,539.0,6.0,1,12669.0,340.0,NaN,1863.0,36.0,Good
4,1503960366,2016-04-19,15506.0,304.0,320.0,0.0,16.0,NaN,NaN,2035.0,50.0,775.0,1.0,0,13019.0,NaN,NaN,1921.0,42.0,Poor
5,1503960366,2016-04-20,10544.0,360.0,377.0,3.0,14.0,NaN,NaN,1786.0,28.0,818.0,2.0,0,15506.0,304.0,NaN,2035.0,50.0,Fair
6,1503960366,2016-04-21,9819.0,325.0,364.0,3.0,36.0,NaN,NaN,1775.0,19.0,838.0,3.0,0,10544.0,360.0,NaN,1786.0,28.0,Poor
7,1503960366,2016-04-23,14371.0,361.0,384.0,1.0,22.0,NaN,NaN,1949.0,41.0,732.0,5.0,1,12764.0,NaN,NaN,1827.0,66.0,Fair
8,1503960366,2016-04-24,10039.0,430.0,449.0,0.0,19.0,NaN,NaN,1788.0,39.0,709.0,6.0,1,14371.0,361.0,NaN,1949.0,41.0,Fair
9,1503960366,2016-04-25,15355.0,277.0,323.0,6.0,40.0,NaN,NaN,2013.0,73.0,814.0,0.0,0,10039.0,430.0,NaN,1788.0,39.0,Poor
